In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import yfinance as yf
import datetime as dt
import plotly.graph_objects as go

today = dt.datetime.today()

nasdaq = pd.read_csv('nasdaq.csv').set_index('Date')

In [200]:
def plotReturns(df, m, s, expRet, volThresh):
    fig = px.scatter(x = df[m],
                    y=df[s],
                    hover_name=df.index)
    
    N=10
    random_x = np.linspace(0, df[m].max(), N)
    random_y = np.linspace(0, df[s].max(),N)
    fig.add_trace(go.Scatter(x=random_x, y=[volThresh]*N,
                    mode='lines',
                    name='lines'))
    fig.add_trace(go.Scatter(x=[expRet]*N, y=random_y,
                    mode='lines',
                    name='lines'))
    fig.show()

In [201]:
def momentumStratVolCutOff(data, 
                           expRet, 
                           volThresh,
                           window,
                           timeFrame,
                           OHLC,
                           end,ticker = 'Ticker'):
    '''
    expRet float
    volRet float
    timeFrame int days
    
    we want to find stocks that within the timeframe selected 
    give us a return greater than or at least the expected return 
    with a std less than the volRet specificed'''
    
    end = pd.to_datetime(end)
    
    timePeriod = end - dt.timedelta(timeFrame)
    
    df = pd.pivot_table(data = data, values = OHLC, columns=ticker,index = 'Date')
    
    df = df[(df.index < str(end))&(df.index > str(timePeriod))]
    
    
    df_returns = df.rolling(window = window).apply(lambda x: (100*(x[-1] - x[0]))/x[0])
    returns = df_returns.agg(['mean', 'std']).T
    
    plotReturns(returns, 'mean', 'std', expRet, volThresh)
    
    return returns

In [204]:
momentumStratVolCutOff(nasdaq, 5, 10, 20, 100, 'Close', '2020-08-01')